In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from tqdm import tqdm
import copy

In [2]:
import pandas as pd

df = pd.read_csv("/data/heart_clean.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../data/heart_clean.csv'

In [ ]:
# Prepare Data with Proper Scaling
X = df.drop('target', axis=1).values
y = df['target'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

# Create datasets and loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

In [ ]:
class HeartDiseaseNet(nn.Module):
    def __init__(self, input_size):
        super(HeartDiseaseNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.bn1 = nn.BatchNorm1d(32)
        self.dropout1 = nn.Dropout(0.2)

        self.fc2 = nn.Linear(32, 16)
        self.bn2 = nn.BatchNorm1d(16)
        self.dropout2 = nn.Dropout(0.2)

        self.fc_out = nn.Linear(16, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout2(x)

        x = self.fc_out(x)
        return x

In [ ]:
# Initialize model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = HeartDiseaseNet(input_size=X_train.shape[1]).to(device)

# class weights
class_counts = np.bincount(y_train)
total = len(y_train)
class_weights = total / (2.0 * class_counts)
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)

print("Class counts (train):", class_counts)
print("Class weights:", class_weights.detach().cpu().numpy())
print(f"Using device: {device}")
print(f"\nModel Architecture:\n{model}")

In [ ]:
def train_function(model, train_loader, test_loader, epochs, optimizer, criterion):

    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []

    best_state = None
    best_test_loss = float('inf')
    patience = 10
    patience_counter = 0

    for epoch in tqdm(range(epochs)):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_total += y_batch.size(0)
            train_correct += (predicted == y_batch).sum().item()

        model.eval()
        test_loss = 0
        test_correct = 0
        test_total = 0

        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)

                test_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                test_total += y_batch.size(0)
                test_correct += (predicted == y_batch).sum().item()

        train_loss /= len(train_loader)
        test_loss /= len(test_loader)
        train_acc = train_correct / train_total
        test_acc = test_correct / test_total

        train_losses.append(train_loss)
        test_losses.append(test_loss)
        train_accuracies.append(train_acc)
        test_accuracies.append(test_acc)

        if test_loss < best_test_loss - 1e-4:
            best_test_loss = test_loss
            best_state = copy.deepcopy(model.state_dict())
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    return train_losses, test_losses, train_accuracies, test_accuracies

In [ ]:
train_losses, test_losses, train_accs, test_accs = train_function(
    model, train_loader, test_loader, epochs=100, optimizer=optimizer, criterion=criterion
)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor.to(device))
    _, preds = torch.max(outputs, 1)
    y_pred = preds.cpu().numpy()

print(classification_report(y_test, y_pred, digits=4))
print(confusion_matrix(y_test, y_pred))

In [ ]:
# Plot Training Metrics
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(train_losses, label='Train Loss', linewidth=2)
axes[0].plot(test_losses, label='Test Loss', linewidth=2)
axes[0].set_title('Training and Validation Loss')
axes[0].legend()

axes[1].plot(train_accs, label='Train Accuracy', linewidth=2)
axes[1].plot(test_accs, label='Test Accuracy', linewidth=2)
axes[1].set_title('Training and Validation Accuracy')
axes[1].legend()

plt.show()

In [ ]:
# Comprehensive Evaluation
model.eval()
all_preds = []
all_labels = []
all_probs = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        probs = F.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(y_batch.numpy())
        all_probs.extend(probs[:, 1].cpu().numpy())

In [ ]:
accuracy = sum(np.array(all_preds) == np.array(all_labels)) / len(all_labels)
print(f'FINAL TEST ACCURACY: {accuracy:.4f}')

In [ ]:
cm = confusion_matrix(all_labels, all_preds)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title('Confusion Matrix')

fpr, tpr, _ = roc_curve(all_labels, all_probs)
roc_auc = auc(fpr, tpr)

axes[1].plot(fpr, tpr, label=f'AUC = {roc_auc:.3f}')
axes[1].plot([0, 1], [0, 1], '--')
axes[1].set_title('ROC Curve')
axes[1].legend()

plt.show()